# Hosts: Generic republishing


This notebook presents the rare case where a manager needs to republish entity data without any knowledge of the provenance of the entity. This may be useful for a generic asset introspection and modification tool. Primarily, this notebook is an exploration of advanced interplay between the entity introspection and publishing APIs.

## Setup

See "Hello OpenAssetIO" notebook for details on how to bootstrap OpenAssetIO. 

In [8]:
from resources import helpers

manager, context = helpers.bootstrap("resources/generic_republish/openassetio_config.toml")

## Getting started

In the following examples we're going to ask the manager about the traits it supports with respect to an asset, and use that to determine a publishing workflow, without knowing anything about the entity.

For these examples, we'll make use of the `project_artwork/logos/openassetio` entity in the asset management system. We must first request an `EntityReference` object representing this entity, for use in API queries:

In [9]:
logo_ref = manager.createEntityReference("bal:///project_artwork/logos/openassetio")

helpers.display_result(repr(logo_ref))

> **Result:**
> `<openassetio.EntityReference bal:///project_artwork/logos/openassetio>`

## Generic resolve and re-publish

A rare use-case is for a tool that blindly resolves, updates, and re-publishes an entity, regardless what type of entity it is. 

Such generic re-publishing is discouraged and dangerous. For example, if "approval" status is blindly resolved and re-published, the manager may not know how it should handle this properly. A better approach to such a generic tool is to present the traits and their properties to the user to select before re-publishing.

However, such workflows are possible and have their place in a pipeline. This can be accomplished by making use of `entityTraits`, alongside other introspection and publishing methods. The following subsections explore two possible workflows, illustrating how generic re-publishing might be achieved. 

### Re-publishing a mutated entity

The following presents an example of blindly updating the display name for any entity. Note that we make use of the fact that managers should silently ignore data that it cannot persist when publishing.

In [10]:
from openassetio.access import EntityTraitsAccess, PolicyAccess, ResolveAccess, PublishingAccess
from openassetio_mediacreation.traits.identity import DisplayNameTrait

# Get the complete trait set of the entity.
entity_trait_set = manager.entityTraits(logo_ref, EntityTraitsAccess.kRead, context)

# Ensure the manager will accept a publish of this entity with an updated display name.
[policy_data] = manager.managementPolicy(
    [entity_trait_set | {DisplayNameTrait.kId}], PolicyAccess.kWrite, context)

if not DisplayNameTrait.kId in policy_data.traitSet():
    raise Exception("Cannot update display name of this entity")

# Get all the properties of the given entity.
data_to_publish = manager.resolve(logo_ref, entity_trait_set, ResolveAccess.kRead, context)

# Any traits without properties, or where the manager cannot provide them, will be missing from the data.
# We still need to imbue those traits, so that manager knows what kind of entity we are publishing.
data_to_publish.addTraits(entity_trait_set)

# Create/update the name of the entity.
DisplayNameTrait(data_to_publish).setName("My New Name")

# Publish it. Any properties we `resolve`d that cannot be re-published will be silently ignored.
updated_ref = manager.register(logo_ref, data_to_publish, PublishingAccess.kWrite, context)

### Re-publishing a minimal entity

The following example dives deeper into the interaction between `entityTraits`, `managementPolicy` and `resolve` for a generic re-publisher. 

We want to re-publish a minimal entity (i.e. only the traits absolutely required for the given entity reference), with two new/updated traits. One of the traits, `BearerTokenTrait`, might not be supported, and the other trait, `LocatableContentTrait`, might have its properties dictated by the manager.

In [11]:
from openassetio_mediacreation.traits.auth import BearerTokenTrait
from openassetio_mediacreation.traits.content import LocatableContentTrait

# The minimum set of traits required to publish to this entity reference.
minimum_trait_set = manager.entityTraits(logo_ref, EntityTraitsAccess.kWrite, context)

# Whatever the minimum trait set is, we know we want to publish a location and auth token.
desired_trait_set = minimum_trait_set | {BearerTokenTrait.kId, LocatableContentTrait.kId}

# Get the set of traits that have properties the manager can persist.
[policy_for_desired_traits] = manager.managementPolicy(
    [desired_trait_set], PolicyAccess.kWrite, context)

# Filter down the desired traits to only those that are supported.
trait_set_to_publish = desired_trait_set & policy_for_desired_traits.traitSet()

# We want to keep (the minimum amount of) data from the previous version, except for the values we're going to
# provide.
trait_set_to_keep = trait_set_to_publish - {BearerTokenTrait.kId, LocatableContentTrait.kId}

# Get the properties that we wish to keep from the current version.
data_to_publish = manager.resolve(logo_ref, trait_set_to_keep, ResolveAccess.kRead, context)

# Any traits without properties, or where the manager cannot provide them, will be missing from the data.
# We still need to imbue those traits, so that manager knows what kind of entity we are publishing.
data_to_publish.addTraits(minimum_trait_set)

# Get the manager's policy for dictating trait properties, i.e. which traits the manager can "drive" for us.
[policy_for_derived_traits] = manager.managementPolicy(
    [trait_set_to_publish], PolicyAccess.kManagerDriven, context)

# Check if the manager can derive a location for us.
if LocatableContentTrait.kId in policy_for_derived_traits.traitSet():
    # Imbue an empty LocatableContentTrait, so that the manager is aware in `preflight` that we intend to publish
    # this trait. We will ask the manager to fill in the value for us before calling `register`.
    LocatableContentTrait.imbueTo(data_to_publish)
else:
    # If the manager doesn't want to provide a location for entities of this type, use a default.
    LocatableContentTrait(data_to_publish).setLocation("file:///tmp/file")

# Manager might not support BearerTokenTrait.
if BearerTokenTrait.kId in trait_set_to_publish:
    # BearerTokenTrait is supported, so imbue and configure.
    BearerTokenTrait(data_to_publish).setToken("==ZxErn43G")

# We can now successfully begin the publishing process.
working_ref = manager.preflight(logo_ref, data_to_publish, PublishingAccess.kWrite, context)

# Check if the manager can provide a location to us.
if LocatableContentTrait.kId in policy_for_derived_traits.traitSet():
    derived_data = manager.resolve(
        working_ref, {LocatableContentTrait.kId}, ResolveAccess.kManagerDriven, context)

    LocatableContentTrait(data_to_publish).setLocation(
        LocatableContentTrait(derived_data).getLocation())
    
# [Do some work to write the new file...]

# We can now finally publish
updated_ref = manager.register(logo_ref, data_to_publish, PublishingAccess.kWrite, context)